### Armington Trade Model

This notebook illustrates how a basic Armington trade model operaet.

In [2]:
include("../src/armington-trade-environment.jl") # loads functions which describe the enviornment
include("../src/armington-trade-solution.jl") # loads functions which can be used to find an equillibrium

using MINPACK # the solver.

#### High-Level Description

1. Environment

The main file that computes trade flows is ``trade_flows`` and it takes endogenous objects (prices, price index, aggregate demand) and then returns a structure ``trade_stats`` which is composed of trade flows in expenditure share form, in value, tariff revenue, world demand, and the price index.  Helper files with parameters, compute the ces price index, all goods prices are in the environment file as well. Two important points:

- All prices are normalized so the country in the first entry's price index is one. 

- Trade shares, flows, and revenue are expressed as a matrix with rows being the buyer dimension and columns being the seller dimension. 

2. Solution file contains ``trade_equilibrium`` which takes endogenous objects (wages, aggregate demand, production factors) it then computes the pattern of trade and world wide demand for each countries commodity and compares it against the production in each country. 

---

#### Computing Trade Flows

In [23]:
trade_params()

(θ = 4.0, τ = [0.0 0.0; 0.0 0.0], d = [1.0 1.5; 1.5 1.0], A = [1.0, 1.0], Ncntry = 2)

Here ``trade_params()`` is a function from the ``Parameters.jl`` package that preloads parameter values. They are set up in the following way:

```
trade_params = @with_kw (
    θ = 4.0, # Armington Elasticity, controls elasticity of substitution in CES model.
    τ = [0.0  0.0; 0.0 0.0], # tariff rate, row = buyer, column = supplier
    d = [1.0  1.5; 1.5 1.0], # trade cost, row = buyer, column = supplier
    A = [1.0, 1.0], #TFP, # Productivity in each country
    Ncntry = length(A),
    
)
```

Then below is a basic call of how to compute the trade flows. 

In [24]:
params = trade_params()

w = [1.0 ,1.0] # the wage rate in each country

N = [1.0 ,1.0] # the wage rate in each country

p, Pindex = goods_prices(w, params )

AD = (w .* N) ./ Pindex 

# this is aggregate demand. Needs to be in local consumption units

trade = trade_flows(p, Pindex, AD, params );

**Note** this is setup a little bit differently than standard trade settings. Here I'm de-linking the notion between income, production, and demand. What this means is wages -> goods prices. Then goods prices + aggregate demand of goods of AD -> where a country satisfies it's demand and by how much. An equillibrium is some notion about how wages (and income more broadly) relate to aggregate demand. The standard trade setting would say that whatever you eat $= \frac{w_{i}*L_{i}}{P_{i}}$ in final consumption units, but here it needs not be that.

In [25]:
trade.trade_share

2×2 Matrix{Float64}:
 0.771429  0.228571
 0.228571  0.771429

This is each countrie expenditure share of goods from different locations. What this says is ``0.77`` in the first row, first column means that country one buys 77 percent of stuff from it self and then the first row, second column means that country one buys 23 percent of it's stuff from country two. 

In [26]:
trade.τ_revenue

2×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0

Here are things expressed in value terms. This is setup so it's in units of the commodity the country produces.

In [27]:
trade.trade_value

2×2 Matrix{Float64}:
 0.771429  0.228571
 0.228571  0.771429

Then because everything is symetric here it turns out to be the same as the shares.

Now this is setup so that we can match up demand with supply. How does this work. Now world demand for a country's commodity is the sum down a column.

In [28]:
trade.world_demand

2-element Vector{Float64}:
 0.9999999999999997
 0.9999999999999997

#### Computing a (Trade) Equillibrium

This is a simple setting, but a trade equillibrium is a wage rate $w$ such that world demand = world supply for each commodity. Pretty simple. This is how it works below in a 2 country, assymetric world.

In [34]:
w = [1.0 ,1.0]

L = [1.0 ,1.0]

d = [1.0 2.0; 3.0 1.0] 
τ = [0.0 0.0; 0.0 0.0]

params = trade_params(d = d, τ = τ)

Ncntry = params.Ncntry

function f!(fvec, x)
    
    foow = x[1:Ncntry] # this is the wage
    
    fooτ = x[Ncntry+1:end] # this is the tariff revenue...
    
    Pindex = goods_prices(foow, params)[2] # get price index so aggregate demand can be in consumption units.

    fvec .= trade_equilibrium(foow, ( foow.*L .+ fooτ)./ Pindex, L, fooτ, params) 
    # returns Ncntry x 2 vector 
    # first Ncntry entries are difference between world demand and world supply
    # next Ncntry entries are difference between conjecture revenue and actuall tariff revenue

end

f! (generic function with 1 method)

In [35]:
initial_x = [1.0 ,1.0 , 0.0, 0.0]

n = length(initial_x)
diag_adjust = n - 1

sol = fsolve(f!, initial_x, show_trace = true, method = :hybr;
      ml=diag_adjust, mu=diag_adjust,
      diag=ones(n),
      mode= 1,
      tol=1e-5,
       )

Iter     f(x) inf-norm    Step 2-norm      Step time
------   --------------   --------------   --------------
     1     1.154387e-01     0.000000e+00         0.017000
     2     1.059735e-02     3.796220e-02         0.000000
     3     1.032742e-03     4.500734e-04         0.000000
     4     3.359724e-05     4.328248e-06         0.001000
     5     3.101237e-06     1.843287e-09         0.000000
     6     2.436241e-08     1.590422e-11         0.000000


Results of Nonlinear Solver Algorithm
 * Algorithm: Modified Powell
 * Starting Point: [1.0, 1.0, 0.0, 0.0]
 * Zero: [1.024270000710426, 1.2166194421736258, 0.0, 0.0]
 * Inf-norm of residuals: 0.000000
 * Convergence: true
 * Message: algorithm estimates that the relative error between x and the solution is at most tol
 * Total time: 0.018000 seconds
 * Function Calls: 6
 * Jacobian Calls (df/dx): 1

In [36]:
wsolution = sol.x

4-element Vector{Float64}:
 1.024270000710426
 1.2166194421736258
 0.0
 0.0

In [38]:
wage = wsolution[1:Ncntry]
    
τ_revenue = wsolution[Ncntry+1:end]

Pindex = goods_prices(wage, params)[2]

trade = trade_equilibrium(wage, (wage.*L) ./ Pindex , L, τ_revenue, params, output = "all")

trade_stats{Float64}([0.9531715425425131 0.0710984581679131; 0.07109845819899586 1.1455209839746303], [0.9305862144565402 0.06941378554345998; 0.05843935723398482 0.9415606427660154], [0.0 0.0; 0.0 0.0], [1.024270000741509, 1.2166194421425434], [1.0000000000000002; 1.1924427204563686;;])

In [39]:
trade.trade_share

2×2 Matrix{Float64}:
 0.930586   0.0694138
 0.0584394  0.941561

So in this setting country one is buys 93 percent from itself, 7 percent from country 2. The assymetry here arises because the country 2 faces a larger trade cost to buy from one than vice versa.

As one check of this, we can compare the real wage vs. the ACR formula... the trade share to the power $1/(1-\theta)$.

In [40]:
(trade.trade_share[1,1])^(1.0 / (1.0 - params.θ))

1.0242700211949347

In [41]:
wage[1]

1.024270000710426

So what is happening here is that the real wage is corresponding with the ACR forumula.

In [43]:
wage[2] / Pindex[2]

1.0202749543458192